In [1]:
import pandas as pd
from glob import glob

In [2]:
scenario_id = "34-1"
scenario_path = "/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware-Capture-"+scenario_id+"/"

In [25]:
!cd {scenario_path}bro && unexpand -t 3 conn.log.labeled > conn.log && sed -i '1,8d' conn.log && sed 's/ //g' -i conn.log && <<< $(cat ../../../base.log) < conn.log | sponge conn.log
!cd {scenario_path}zeek && sed -i '1,8d' flowmeter.log && sed 's/ //g' -i flowmeter.log && <<< $(cat ../../../base_zeek.log) < flowmeter.log | sponge flowmeter.log

In [3]:
conn = pd.read_csv(scenario_path + 'bro/conn.log', sep="\t", header=0)
conn.drop(conn.shape[0]-1, inplace=True)
conn = conn[["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state", "label", "detailed-label"]]
conn.head(1)

,ts,uid,id.orig_h,id.resp_h,id.resp_p,proto,service,conn_state,label,detailed-label
0,1545403816.962094,CrDn63WjJEmrWGjqf,192.168.1.195,185.244.25.235,80.0,tcp,-,S0,Benign,-


In [4]:
flow = pd.read_csv(scenario_path + 'zeek/flowmeter.log', sep="\t", header=0, low_memory=False, index_col=False)
flow.drop(flow.shape[0]-1, inplace=True)
flow.drop("uid", axis=1, inplace=True)
flow.head(1)

,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size
0,3.139211,3.0,0.0,0.0,0.0,0.955654,0.0,0.955654,0.0,120.0,...,0.0,0.0,0.0,0.0,0.0,0.0,29200.0,0.0,29200.0,0.0


In [5]:
dataset = pd.concat([conn, flow], axis=1)
dataset.drop(["ts", "uid", "id.orig_h", "id.resp_h", "id.resp_p", "proto", "service", "conn_state"], axis=1, inplace=True)
dataset.head(1)

,label,detailed-label,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,...,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size
0,Benign,-,3.139211,3.0,0.0,0.0,0.0,0.955654,0.0,0.955654,...,0.0,0.0,0.0,0.0,0.0,0.0,29200.0,0.0,29200.0,0.0


In [6]:
benign = dataset[dataset["label"] == "Benign"]
benign.to_csv('/media/kmdr7/Seagate/DATASETS/Merged-IoT-Dataset/Benign/'+scenario_id+'.csv', index=False)
benign.shape

(1923, 82)

In [7]:
malware = dataset[dataset["label"] != "Benign"]
malware.to_csv('/media/kmdr7/Seagate/DATASETS/Merged-IoT-Dataset/Malware/'+scenario_id+'.csv', index=False)
malware.shape

(21222, 82)